<a href="https://colab.research.google.com/github/prithvijaunjale/NLP/blob/master/sentiment_analysis/sentiment_analysis_cnn_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip

--2020-04-27 16:56:06--  http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip [following]
--2020-04-27 16:56:06--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘trainingandtestdata.zip’

trainingandtestdata 100%[===================>]  77.59M  20.1MB/s    in 4.8s    

2020-04-27 16:56:11 (16.0 MB/s) - ‘trainingandtestdata.zip’ saved [81363704/81363704]



In [ ]:
!unzip trainingandtestdata.zip

Archive:  trainingandtestdata.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import pickle
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [ ]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
train_data = pd.read_csv(
    "train.csv",
    header=None,
    names=cols,
    engine="python",    #avoid charset errors
    encoding="latin1"   #standard for western languages
)
test_data = pd.read_csv(
    "test.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [ ]:
train_data.head(3)

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [ ]:
data = train_data
data.drop(["id", "date", "query", "user"], # don't forget to run data = train_data before!
          axis=1,
          inplace=True)

def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Removing the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Removing the URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Keeping only letters
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Removing additional whitespaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

data_clean = [clean_tweet(tweet) for tweet in data.text]

data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

In [ ]:
data_clean[:5]

[" Awww that's a bummer. You shoulda got David Carr of Third Day to do it. D",
 "is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!",
 ' I dived many times for the ball. Managed to save The rest go out of bounds',
 'my whole body feels itchy and like its on fire ',
 " no it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "]

In [ ]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size=2**16
)

data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

In [ ]:
MAX_LEN = max([len(sentence) for sentence in data_inputs])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

In [ ]:
# SAVE
drive_path = 'drive/My Drive/deep_learning/nlp/cnn_sentiment_analysis/'
with open(drive_path + 'data_clean.pkl', 'wb') as f:
    pickle.dump(data_clean, f)
with open(drive_path + 'data_labels.pkl', 'wb') as f:
    pickle.dump(data_labels, f)
with open(drive_path + 'tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(drive_path + 'data_inputs.pkl', 'wb') as f:
    pickle.dump(data_inputs, f)

In [ ]:
# LOAD
drive_path = 'drive/My Drive/deep_learning/nlp/cnn_sentiment_analysis/'
with open(drive_path + 'data_clean.pkl', 'rb') as f:
    data_clean = pickle.load(f)
with open(drive_path + 'data_labels.pkl', 'rb') as f:
    data_labels = pickle.load(f)
with open(drive_path + 'tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
with open(drive_path + 'data_inputs.pkl', 'rb') as f:
    data_inputs = pickle.load(f)

In [ ]:
dt_labels = pd.get_dummies(data_labels).values
# Instead you can use sparse_categorical_crossentropy as loss fn (i.e. for integers)
# & metrics=['sparse_categorical_accuracy']

# else for OneHotEncoded labels use categorical_crossentropy
dt_labels[:5]

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [ ]:
from sklearn.model_selection import train_test_split

# test_idx = np.random.randint(0, 800000, 8000)
# test_idx = np.concatenate((test_idx, test_idx+800000))

# test_inputs = data_inputs[test_idx]
# test_labels = dt_labels[test_idx]
# train_inputs = np.delete(data_inputs, test_idx, axis=0)
# train_labels = np.delete(dt_labels, test_idx)

train_inputs, test_inputs, train_labels, test_labels = train_test_split(data_inputs, dt_labels, test_size=0.005, random_state=42)
print(len(train_inputs), len(train_labels))
print(len(test_inputs), len(test_labels))

1592000 1592000
8000 8000


Model

In [ ]:
class DCNN(tf.keras.Model):
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name='dcnn'):
        super(DCNN, self).__init__(name=name)

        self.embedding = layers.Embedding(vocab_size, emb_dim)

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding='valid',
                                    activation='relu')
        self.trigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=3,
                                    padding='valid',
                                    activation='relu')
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=4,
                                    padding='valid',
                                    activation='relu')
        self.pool = layers.GlobalMaxPool1D()

        self.dense_1 = layers.Dense(units=FFN_units)
        self.dropout = layers.Dropout(rate=dropout_rate)
        
        self.last_dense = layers.Dense(units=nb_classes, activation='softmax')

    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)

        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

Config

In [ ]:
VOCAB_SIZE = tokenizer.vocab_size
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2
DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

In [ ]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [ ]:
Dcnn.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
checkpoint_path = drive_path + 'ckpt/'

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!")

In [ ]:
Dcnn.fit(train_inputs,
         train_labels,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS,
         validation_split=0.005)
ckpt_manager.save()

Epoch 1/5
49502/49502 [==============================] - 5552s 112ms/step - loss: 0.4073 - accuracy: 0.8154 - val_loss: 0.3731 - val_accuracy: 0.8315
Epoch 2/5
 7683/49502 [===>..........................] - ETA: 1:18:50 - loss: 0.3382 - accuracy: 0.8553Buffered data was truncated after reaching the output size limit.

In [ ]:
ckpt_manager.save()

'drive/My Drive/deep_learning/nlp/cnn_sentiment_analysis/ckpt/ckpt-1'